# Load a prepared SAMPL6 system


This notebook downloads a SAMPL6 challenge system as an XML-serialized OpenMM System, creates an `Interchange` from it, and then computes the energy of the starting coordinates with GROMACS.

In [ ]:
from openff.interchange import Interchange
from openff.interchange.drivers import get_summary_data

In [ ]:
!wget -O complex.pdb https://raw.githubusercontent.com/samplchallenges/SAMPL6/c661d3985af7fa0ba8c64a1774cfb2363cd31bda/host_guest/SAMPLing/CB8-G3-3/PDB/complex.pdb
!wget -O complex.xml https://raw.githubusercontent.com/samplchallenges/SAMPL6/c661d3985af7fa0ba8c64a1774cfb2363cd31bda/host_guest/SAMPLing/CB8-G3-3/OPENMM/complex.xml

In [ ]:
import openmm
import openmm.app

pdb_object = openmm.app.PDBFile("complex.pdb")

topology = pdb_object.topology
positions = pdb_object.positions

system = openmm.XmlSerializer.deserialize(open("complex.xml").read())

# Remove force objects that are not stored in Interchange and do not affect the energy
for force_index in range(system.getNumForces())[::-1]:
    if isinstance(
        system.getForce(force_index),
        openmm.MonteCarloBarostat | openmm.CMMotionRemover,
    ):
        system.removeForce(force_index)

In [ ]:
imported = Interchange.from_openmm(
    system=system,
    topology=pdb_object.topology,
    positions=pdb_object.getPositions(),
)

# Residue numbers in this PDB start at 0, but GROMACS requires that
# residue numbers start at 1, so modify them in-place
for atom in imported.topology.atoms:
    atom.metadata["residue_number"] = str(int(atom.metadata["residue_number"]) + 1)

In [ ]:
# Interchange.to_amber is not yet optimized for biopolymer systems
summary = get_summary_data(imported, _engines=["GROMACS", "OpenMM"])
summary